In [59]:
%pip install gradio

6867.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [3]:
import gradio as gr

/private/var/folders/dd/jlnky4852dq6trt16wfd4rc80000gn/T/gradio/af57fca847398ba41d59b9150f42049dbe036586/audio.wav
{'RecognitionStatus': 'Success', 'Offset': 7500000, 'Duration': 20800000, 'NBest': [{'Confidence': 0.9173006, 'Lexical': '남해 여행지 알려줘 알 수 있을 것 같아요', 'ITN': '남해 여행지 알려줘 알 수 있을 것 같아요', 'MaskedITN': '남해 여행지 알려줘 알 수 있을 것 같아요', 'Display': '남해 여행지 알려줘. 알 수 있을 것 같아요?'}], 'DisplayText': '남해 여행지 알려줘.알 수 있을 것 같아요?'}
None
/private/var/folders/dd/jlnky4852dq6trt16wfd4rc80000gn/T/gradio/71e96c549ddabd36bedd8174694716b6124531e5/audio.wav
{'RecognitionStatus': 'Success', 'Offset': 5900000, 'Duration': 13600000, 'NBest': [{'Confidence': 0.9211209, 'Lexical': '남해 여행지 알려줘', 'ITN': '남해 여행지 알려줘', 'MaskedITN': '남해 여행지 알려줘', 'Display': '남해 여행지 알려줘.'}], 'DisplayText': '남해 여행지 알려줘.'}
남해 여행지 알려줘. []
{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False

In [5]:
import gradio as gr
import requests
import numpy as np
import re

# Azure OpenAI 설정 (여기에 실제 엔드포인트와 API 키를 넣어주세요)  
AZURE_OPENAI_ENDPOINT = "https://fimtrus-openai.openai.azure.com/"
AZURE_OPENAI_API_KEY = "310a6832c2394daf97a3f446cc86ce20"
DEPLOYMENT_NAME = "fitmrus-gpt4o"

# Azure OpenAI API 호출 함수  
def chatgpt_response(prompt, history):  
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # Payload for the request

    messages = []

    #System
    messages.append(
        
        {
        "role": "system",
        "content": "너는 남해 여행 전문가야."
        },
    )

    #assistant
    if len(history) > 0:
        for text in history[0]:
            messages.append({
                "role": "assistant",
                "content": text            
            })

    #user
    messages.append({
            "role": "user",
            "content": prompt
        })

    payload = {
        "messages": messages,
        "temperature": 0,
        "top_p": 0.95,
        "max_tokens": 800,
        # "azureSearchEndpoint": "https://fimtrus-ai-search2.search.windows.net",
        # "azureSearchKey": "oMVRj7m3JohAluhSM4N8kPhDDQiMw6ZMDBsOcnVbi6AzSeDj6vvI",
        # "azureSearchIndexName": "travel-index",
        # "data_sources": [
        # {
        #     "type": "azure_search",
        #     "parameters": {
        #         "endpoint": "https://fimtrus-ai-search2.search.windows.net",
        #         "index_name": "travel-index",
        #         "semantic_configuration": "travel-semantic",
        #         "query_type": "semantic",
        #         "fields_mapping": {},
        #         "in_scope": False,
        #         "role_information": "",
        #         "filter": None,
        #         "strictness": 3,
        #         "top_n_documents": 5,
        #         "authentication": {
        #         "type": "api_key",
        #         "key": "oMVRj7m3JohAluhSM4N8kPhDDQiMw6ZMDBsOcnVbi6AzSeDj6vvI"
        #         },
        #         "key": "oMVRj7m3JohAluhSM4N8kPhDDQiMw6ZMDBsOcnVbi6AzSeDj6vvI",
        #         "indexName": "travel-index"
        #     }
        # }
        # ]
    }
  
    response = requests.post(  
        f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",  
        headers=headers,  
        json=payload  
    )  
    
    result = response.json()  

    if result and result['choices']:
        bot_response = result['choices'][0]['message']['content'].strip()
        bot_response = re.sub(r'\[doc(\d+)\]', r'[\1]', bot_response)
    else:
        return '', '', []

    history.append((prompt, bot_response))  
    citation_list = []

    # if result['choices'][0]['message']['context']:
    #     citations = result['choices'][0]['message']['context']['citations']
    #     i = 0
    #     for citation in citations:
    #         i += 1
    #         html = f"<details><summary>참조{i}</summary><ul>{citation['content']}</ul></details>"
    #         citation_list.append(html)
            # citation_list.append(gr.Textbox(citation['content']))
    return '', history, ''.join(citation_list)


def greet(name):
    name_length = len(name)
    if name_length > 5:
        return "Hello!!! " + name + " World!!!"
    else:
        return ""

def select_choice(value):
    return value, value

question_list = [
    ("남해 2박 3일", "남해 2박3일 여행을 하려고하는데 관광지 추천해줘"),
    ("남해 맛집", "남해 맛집 추천해줘"),
    ("남해 친구 3박 4일", "남해 친구랑 3박 4일 여행가기 좋은 곳은 어디 있을까?"),
]

def request_stt(audio_path):
    # Azure OpenAI 설정
    endpoint = "https://eastus.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1?language=ko-KR&format=detailed"
    api_key = "8e5931bade634c4e859a8e7544f87ff7"

    if audio_path is None:
        return ''
    
    with open(audio_path, "rb") as audio:
        audio_data = audio.read()

    response = requests.post(endpoint, 
                             headers={
                                # "Accept": "application/json",
                                'Content-Type': 'audio/wav',
                                "Ocp-Apim-Subscription-Key":api_key  
                             }, 
                             data=audio_data
    )

    response_json = response.json()
    print(response_json)
    status = response_json.get('RecognitionStatus', None)
    if status == 'Success':
        text = response_json.get('NBest')[0].get('Display')
        return text
    else:
        return ''

def change_audio(audio_path):
    print(audio_path)
    
    # request STT
    text = request_stt(audio_path=audio_path)
    return text

# TTS
def get_token():
    subscription_key = '8e5931bade634c4e859a8e7544f87ff7'
    fetch_token_url = 'https://eastus.api.cognitive.microsoft.com/sts/v1.0/issueToken'
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key
    }
    response = requests.post(fetch_token_url, headers=headers)
    access_token = str(response.text)
    return access_token

def request_tts(text, file_name='response_audio.wav'):

    # Azure OpenAI 설정
    endpoint = "https://eastus.tts.speech.microsoft.com/cognitiveservices/v1"
    api_key = "8e5931bade634c4e859a8e7544f87ff7"
    token = get_token()
    data = f"""<speak version='1.0' xml:lang='ko-KR'><voice xml:lang='ko-KR' xml:gender='Female'
        name='ko-KR-JiMinNeural'>
            {text}
    </voice></speak>"""
    response = requests.post(endpoint, 
                             headers={
                                # "Accept": "application/json",
                                'Authorization': 'Bearer {}'.format(token),
                                # 'Content-Type': 'application/json',
                                'Content-Type': 'application/ssml+xml',
                                'User-Agent': 'testForEducation',
                                'X-Microsoft-OutputFormat': 'riff-24khz-16bit-mono-pcm',
                                # 'X-Microsoft-OutputFormat': 'audio-24khz-16bit-24kbps-mono-opus',
                                # "Ocp-Apim-Subscription-Key":api_key  
                             }, 
                             data=data
    )

    if response.status_code == 200:
        with open(file_name, 'wb') as audio_file:  
            audio_file.write(response.content)  
        return file_name
    else:
        return None
    
def change_tts(text):
    file_name = request_tts(text)
    if file_name:
        return file_name
    else:
        return ''
    
def change_chatbot(chatbot):
    import re
    text = chatbot[0][-1]
    pattern = r'[^가-힣a-zA-Z0-9\s]'  
    cleaned_text = re.sub(pattern, '', text)  
    file_name = request_tts(cleaned_text, file_name='gpt_response.wav')
    if file_name:
        return file_name
    else:
        return ''        

with gr.Blocks() as demo:

    gr.Markdown("# Fimrus's AI World!!!!")

    with gr.Row():
        with gr.Column(scale=3):
            
            chatbot = gr.Chatbot()
            citation = gr.HTML(label="참조")
            choice = gr.Dropdown(choices=question_list, label="샘플")
            with gr.Column():
                with gr.Row():
                    prompt = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요", scale=5)
                    send_button = gr.Button('전송', scale=1)
                    clear_button = gr.Button('지우기', scale=1)
            send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
            prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
            clear_button.click(lambda: None, None, chatbot, queue=False)
            choice.select(fn=select_choice, inputs=[choice], outputs=[choice, prompt])

            # 자동 tts
            chatbot_audio = gr.Audio(label='GPT', autoplay=True)
            chatbot.change(fn=change_chatbot, inputs=[chatbot], outputs=[chatbot_audio])
          
        with gr.Column(scale=1):
            gr.Markdown('<h3>STT</h3>')
            input_mic = gr.Audio(label="마이크 입력", sources="microphone", type="filepath", waveform_options=gr.WaveformOptions(
                waveform_color="#01C6FF",
                waveform_progress_color="#0066B4",
                skip_length=2,
                show_controls=False,
            ))
            input_file = gr.Audio(label="파일 입력", sources="upload", type="filepath")

            input_mic.change(fn=change_audio, inputs=[input_mic], outputs=[prompt])
            input_file.change(fn=change_audio, inputs=[input_file], outputs=[prompt])

            gr.Markdown('<h3>TTS</h3>')

            tts_textbox = gr.Textbox(label="TTS", placeholder="텍스트를 입력하세요")
            send_tts_button = gr.Button('전송')
            output_tts_audio = gr.Audio(interactive=False)
            send_tts_button.click(fn=change_tts, inputs=[tts_textbox], outputs=[output_tts_audio])
            prompt.submit(fn=change_tts, inputs=[tts_textbox], outputs=[output_tts_audio])
            
demo.launch()

Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


TTS []


In [11]:
text = """
삼동면 동부대로:
이곳은 그루의 방풍림으로 조성된 울창한 수림이 있으며, 남해 바다의 정취를 느낄 수 있는 곳입니다 [doc1].
상주면 보리암로:
금산은 한려해상 국립공원 내의 유일한 산악공원으로, 다양한 전설을 담고 있는 기암괴석이 있습니다. 이곳에서 남해의 아름다운 경치를 감상할 수 있습니다 [doc2].
서면 남서대로:
남해의 최고 높이를 자랑하는 망운산이 위치해 있으며, 금산과 함께 남해를 찾는 손님들의 산으로 유명합니다. 망운산은 남해인들이 가장 아끼는 아름다운 기상을 자랑합니다 [doc3].
남면 남서대로:
아난티 남해 리조트가 위치해 있으며, 이곳에서 편안한 휴식을 취할 수 있습니다 [doc4].
설천면 설천로:
남해 상상양떼목장이 위치해 있으며, 이곳에서 양들과 함께하는 특별한 경험을 할 수 있습니다 [doc5].
이 외에도 남해에는 다양한 자연경관과 문화유산이 많으니, 여행 계획에 참고하시기 바랍니다.
"""

# 원시 문자열로 지정하여 백슬래시를 이스케이프 처리하지 않음
new_text = re.sub(r'\[doc(\d+)\]', r'[\1]', text)

print(new_text)


삼동면 동부대로:
이곳은 그루의 방풍림으로 조성된 울창한 수림이 있으며, 남해 바다의 정취를 느낄 수 있는 곳입니다 [1].
상주면 보리암로:
금산은 한려해상 국립공원 내의 유일한 산악공원으로, 다양한 전설을 담고 있는 기암괴석이 있습니다. 이곳에서 남해의 아름다운 경치를 감상할 수 있습니다 [2].
서면 남서대로:
남해의 최고 높이를 자랑하는 망운산이 위치해 있으며, 금산과 함께 남해를 찾는 손님들의 산으로 유명합니다. 망운산은 남해인들이 가장 아끼는 아름다운 기상을 자랑합니다 [3].
남면 남서대로:
아난티 남해 리조트가 위치해 있으며, 이곳에서 편안한 휴식을 취할 수 있습니다 [4].
설천면 설천로:
남해 상상양떼목장이 위치해 있으며, 이곳에서 양들과 함께하는 특별한 경험을 할 수 있습니다 [5].
이 외에도 남해에는 다양한 자연경관과 문화유산이 많으니, 여행 계획에 참고하시기 바랍니다.

